In [ ]:
import numpy

import tensorflow
import random

In [9]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
import json


json_file_path = "/content/intents.json"

with open(json_file_path) as json_file:
    data = json.load(json_file)


In [11]:
words = []
labels = []
docs_x = []
docs_y = []

In [12]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [13]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [14]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

In [16]:
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=a6f381c5e8399b87389ed4ee68a1595b8e5a28699b1ac506fe7833af19ae0709
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [17]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(8, input_shape=(len(training[0]),)),
    Dense(8),
    Dense(len(output[0]), activation="softmax")
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(training, output, epochs=1000, batch_size=8)

model.save('model.tflearn')


Train on 26 samples
Epoch 1/1000
26/26 [==============================] - 0s 9ms/sample - loss: 1.7451 - acc: 0.3077
Epoch 2/1000
26/26 [==============================] - 0s 547us/sample - loss: 1.7246 - acc: 0.3462
Epoch 3/1000
26/26 [==============================] - 0s 701us/sample - loss: 1.7090 - acc: 0.3462
Epoch 4/1000
26/26 [==============================] - 0s 455us/sample - loss: 1.6939 - acc: 0.3462
Epoch 5/1000
26/26 [==============================] - 0s 455us/sample - loss: 1.6792 - acc: 0.3462
Epoch 6/1000
26/26 [==============================] - 0s 431us/sample - loss: 1.6643 - acc: 0.3846
Epoch 7/1000
26/26 [==============================] - 0s 339us/sample - loss: 1.6498 - acc: 0.3846
Epoch 8/1000
26/26 [==============================] - 0s 338us/sample - loss: 1.6356 - acc: 0.3846
Epoch 9/1000
26/26 [==============================] - 0s 363us/sample - loss: 1.6215 - acc: 0.3846
Epoch 10/1000
26/26 [==============================] - 0s 413us/sample - loss: 1.6080 - acc

In [28]:

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, epochs=1000, batch_size=8)
    model.save("model.tflearn")

Train on 26 samples
Epoch 1/1000
26/26 [==============================] - 0s 489us/sample - loss: 6.2248e-04 - acc: 1.0000
Epoch 2/1000
26/26 [==============================] - 0s 399us/sample - loss: 6.2066e-04 - acc: 1.0000
Epoch 3/1000
26/26 [==============================] - 0s 400us/sample - loss: 6.1863e-04 - acc: 1.0000
Epoch 4/1000
26/26 [==============================] - 0s 493us/sample - loss: 6.1703e-04 - acc: 1.0000
Epoch 5/1000
26/26 [==============================] - 0s 429us/sample - loss: 6.1497e-04 - acc: 1.0000
Epoch 6/1000
26/26 [==============================] - 0s 447us/sample - loss: 6.1282e-04 - acc: 1.0000
Epoch 7/1000
26/26 [==============================] - 0s 419us/sample - loss: 6.1096e-04 - acc: 1.0000
Epoch 8/1000
26/26 [==============================] - 0s 419us/sample - loss: 6.0934e-04 - acc: 1.0000
Epoch 9/1000
26/26 [==============================] - 0s 412us/sample - loss: 6.0735e-04 - acc: 1.0000
Epoch 10/1000
26/26 [==============================] 

In [35]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

import tensorflow as tf
import numpy as np
import random


model = tf.keras.models.load_model('model.tflearn')

def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        processed_input = bag_of_words(inp, words)
        processed_input = np.array([processed_input])


        results = model.predict(processed_input)
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()


Start talking with the bot (type quit to stop)!
You: hi
Good to see you again!
You: hello
Hello!
You: what you doing
Hello!
You: quit
